In [8]:
import numpy as np
import pandas as pd
import openmdao.api as om
import aviary
import aviary.api as av
import aviary.subsystems as avs
from openmdao.utils.options_dictionary import OptionsDictionary
from aviary.variable_info.variables import Aircraft
from standard_evaluator import get_interface, show_structure, create_problem, get_state, set_state, load_state, save_state
from standard_evaluator import save_assembly, load_assembly


from aviary.subsystems.geometry.flops_based.prep_geom import PrepGeom

In [9]:
# Create an empty group
model = om.Group()

prep = PrepGeom()
model.add_subsystem('prep_geom', prep, promotes=['*'])

prob = om.Problem(model)

prob.setup()

# Load values for all variables from a csv file, and then set them
new_df = pd.read_csv('input.csv')
# Drop the unused column
new_df = new_df.drop(["Unnamed: 0"], axis=1)
for (name, val) in (new_df.iloc[0]).items():
    prob.set_val(name, val=np.array(	[val]	))

help_dict = {Aircraft.Propulsion.TOTAL_NUM_FUSELAGE_ENGINES: 1}
prob.model._subsystems_myproc[1]._subsystems_myproc[4].options.set(**help_dict)
help_dict = {Aircraft.Engine.NUM_ENGINES: np.array([2])}
prob.model._subsystems_myproc[1]._subsystems_myproc[6].options.set(**help_dict)
prob.model._subsystems_myproc[1]._subsystems_myproc[8].options.set(**help_dict)

# Run the model
prob.run_model()
_ = prob.model.list_inputs(shape=True)


78 Input(s) in 'model'

varname                                              val              shape  prom_name                                      
---------------------------------------------------  ---------------  -----  -----------------------------------------------
prep_geom
  fuselage_prelim
    aircraft:fuselage:length                         [120.]           (1,)   aircraft:fuselage:length                       
    aircraft:fuselage:max_height                     [0.]             (1,)   aircraft:fuselage:max_height                   
    aircraft:fuselage:max_width                      [30.]            (1,)   aircraft:fuselage:max_width                    
  wing_prelim
    aircraft:wing:area                               [2.873]          (1,)   aircraft:wing:area                             
    aircraft:wing:glove_and_bat                      [0.5]            (1,)   aircraft:wing:glove_and_bat                    
    aircraft:wing:span                               [12.] 

In [10]:
om.n2(prob, 'before.n2.html', display_in_notebook=False,  )
#om.n2(prob, 'before.n2.html', )

In [11]:
info = get_interface(prob.model)
state = get_state(prob.model, info)


save_assembly(prob, assembly_name='demo_group.json', state_name='state.h5')

# Now loading the structure and state from these files
new_problem = load_assembly(assembly_name='demo_group.json', state_name='state.h5')
_ = new_problem.model.list_inputs(shape=True, units=True)

new_problem.run_model()

** This is a group, special handling needed for prep_geom in group 
Adding fuselage_prelim component
Building class fuselage_prelim
Importing class FuselagePrelim from aviary.subsystems.geometry.flops_based.fuselage
Adding wing_prelim component
Building class wing_prelim
Importing class WingPrelim from aviary.subsystems.geometry.flops_based.wing
Adding prelim component
Building class prelim
Importing class _Prelim from aviary.subsystems.geometry.flops_based.prep_geom
Adding wing component
Building class wing
Importing class _Wing from aviary.subsystems.geometry.flops_based.prep_geom
Adding tail component
Building class tail
Importing class _Tail from aviary.subsystems.geometry.flops_based.prep_geom
Adding fuselage component
Building class fuselage
Importing class _Fuselage from aviary.subsystems.geometry.flops_based.prep_geom
Adding nacelles component
Building class nacelles
Importing class Nacelles from aviary.subsystems.geometry.flops_based.nacelle
Adding canard component
Building cl

In [12]:
_ = new_problem.model.list_inputs(shape=True)

78 Input(s) in 'model'

varname                                              val              shape  prom_name                                      
---------------------------------------------------  ---------------  -----  -----------------------------------------------
prep_geom
  fuselage_prelim
    aircraft:fuselage:length                         [120.]           (1,)   aircraft:fuselage:length                       
    aircraft:fuselage:max_height                     [0.]             (1,)   aircraft:fuselage:max_height                   
    aircraft:fuselage:max_width                      [30.]            (1,)   aircraft:fuselage:max_width                    
  wing_prelim
    aircraft:wing:area                               [2.873]          (1,)   aircraft:wing:area                             
    aircraft:wing:glove_and_bat                      [0.5]            (1,)   aircraft:wing:glove_and_bat                    
    aircraft:wing:span                               [12.] 

In [13]:
_ = new_problem.model.list_outputs(shape=True)

37 Explicit Output(s) in 'model'

varname                                             val              shape  prom_name                                     
--------------------------------------------------  ---------------  -----  ----------------------------------------------
prep_geom
  fuselage_prelim
    aircraft:fuselage:avg_diameter                  [15.]            (1,)   aircraft:fuselage:avg_diameter                
    aircraft:fuselage:planform_area                 [3600.]          (1,)   aircraft:fuselage:planform_area               
  wing_prelim
    aircraft:wing:aspect_ratio                      [60.68268015]    (1,)   aircraft:wing:aspect_ratio                    
  prelim
    prep_geom:_Names:CROOT                          [0.21972222]     (1,)   prelim.prep_geom:_Names:CROOT                 
    prep_geom:_Names:CROOTB                         [0.16479167]     (1,)   prelim.prep_geom:_Names:CROOTB                
    prep_geom:_Names:CROTM                          [0